In [1]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
from dcicutils import ff_utils
from functions.notebook_functions import *
import json
import glob
import os

insert_folder = '/Users/koray/Github/fourfront/src/encoded/tests/data/master-inserts'
files = glob.glob(insert_folder+'/*.json')

# limit to the ones you want to use
files = [i for i in files if 'file_format' in i]

store = {}
for a_file in files:
    temp = open(a_file, 'r')
    f_name = os.path.basename(a_file)
    f_name_head = f_name[:-5]
    store[f_name_head] = json.load(temp)
    temp.close()

for i in store:
    print(i, len(store[i]))

file_format 34


In [4]:
only_post = False
posted = 0
transfer_env = 'fourfront-webdev'
transfer_key = ff_utils.get_authentication_with_server({}, ff_env=transfer_env)
schema_name = get_schema_names(transfer_key) 
# reverse lookup dictionary for schema names
rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]
second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=transfer_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    counter=0
    print(len(store[a_type]), 'items exist on source')
    for an_item in store[a_type]:
        counter += 1
        #print(counter, an_item['uuid'])
        # does the item exist
        exists = False
        try:
            # TODO check with all identifiers
            existing = ff_utils.get_metadata(an_item['uuid'], key=transfer_key)
            exists = True
        except:
            exists = False
        if only_post:
            # skip the items that exists
            if exists and existing:
                remove_existing_items.append(an_item['uuid'])
                print("{} {} can not post item".format(obj_type, an_item['file_format']))
                continue
        if not exists:
            posted += 1
            post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
            #ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(posted, 'items posted to target')
    print(len(second_round_items[a_type]), 'items will be patched in second round')
    print()

    

file_format
34 items exist on source
0 items posted to targe
34 items will be patched in second round



In [5]:
# Round II - patch the rest of the metadata
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        continue 
    for an_item in second_round_items[a_type]:
        counter += 1
        ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key)